In [48]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, make_scorer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from category_encoders import TargetEncoder

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from category_encoders import TargetEncoder
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt



In [35]:
X_train = pd.read_csv("X_train_tran.csv")
y_train = pd.read_csv("y_train_tran.csv")
y_train = np.ravel(y_train)
X_val = pd.read_csv("X_val_tran.csv")
y_val = pd.read_csv("y_val_tran.csv")
myColumns = X_train.columns
print(X_val.shape)
print(y_val.shape)
print(X_val.shape)
print(y_val.shape)

df = pd.read_csv("Customer-Churn-Records.csv")
df = df.drop(columns=['RowNumber','CustomerId'])
df = df.drop(columns=['Complain'])
df = df.rename(columns={'Card Type': 'CardType'})
df = df.rename(columns={'Point Earned': 'PointsEarned'})
df = df.rename(columns={'Geography': 'Country'})
df = df.rename(columns={'Satisfaction Score': 'SatisfactionScore'})
# save original columns
myColumns = df.columns

X = pd.read_csv("X.csv")
X_train_cluster = pd.read_csv("X_train_cluster.csv")
X_val_cluster = pd.read_csv("X_val_cluster.csv")
X_test_cluster = pd.read_csv("X_test_cluster.csv")


XX = df.drop(['Exited','Surname'], axis=1) # I drop Surname since we have not encoded it
y = df['Exited']
# yy = np.ravel(y)
X_trainDF, X_testDF, y_trainDF, y_testDF = train_test_split(XX, y, test_size=0.2, random_state=40)

(1000, 15)
(1000, 1)
(1000, 15)
(1000, 1)


In [36]:
myColumns

Index(['Surname', 'CreditScore', 'Country', 'Gender', 'Age', 'Tenure',
       'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Exited', 'SatisfactionScore', 'CardType',
       'PointsEarned'],
      dtype='object')

In [3]:
# from sklearn.model_selection import train_test_split

# # Split the dataset into training, temporary holdout, and final holdout sets
# X_train, X_temp, y_train, y_temp = train_test_split(df, df['Exited'], test_size=0.2, stratify=df['Exited'], random_state=42)

# # Split the temporary holdout set into validation and final holdout sets
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

# print(X_train.shape)
# print(y_train.shape)
# print(X_test.shape)
# print(y_test.shape)
# print(X_val.shape)
# print(y_val.shape)


(8000, 18)
(8000,)
(1000, 18)
(1000,)
(1000, 18)
(1000,)


In [ ]:
"Random Forest": RandomForestRegressor()
"Decision Tree": DecisionTreeRegressor()
"Gradient Boosting": GradientBoostingRegressor()
"Linear Regression": LinearRegression()
"XGBRegressor": XGBRegressor()
"CatBoosting Regressor": CatBoostRegressor(verbose=False)
"AdaBoost Regressor": AdaBoostRegressor()

In [42]:


# Create an instance of GradientBoostingClassifier
model = GradientBoostingClassifier()

# Define the parameter grid to search over
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.1, 0.05, 0.01],
    'max_depth': [3, 4, 5]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(model, param_grid, cv=5, scoring=make_scorer(f1_score, greater_is_better=False), verbose=2, n_jobs=-1)

print(y_train.shape)
y_train = np.ravel(y_train)
print(y_train.shape)
# Fit the GridSearchCV object to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score (F-1):", -grid_search.best_score_)



(8000, 1)
(8000,)
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Best Parameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}
Best Score (F-1): 0.39348612904550817


In [43]:
# X_train_cat = X_train[['GenderBinary', 'CountryOrd','SurnameOrd','HasCrCard','IsActiveMember']]

# X_train_num = X_train[['Tenure', 'NumOfProducts','Satisfaction Score', 'PointsEarnedQuant','AgeOrd','TaxBracket','BalanceOrd',]]

target_enc_features = ['GenderBinary', 'CountryOrd','SurnameOrd','HasCrCard','IsActiveMember', 'Tenure', 'NumOfProducts','SatisfactionScore', 'PointsEarnedQuant','AgeOrd','TaxBracket','BalanceOrd']



# Define the preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('target', TargetEncoder(), target_enc_features),
    ])

# Adjusts weights so that the undersampled class get higher weight value
class_weights = dict(zip(np.unique(y_train), [len(y_train) / (len(np.unique(y_train)) * np.bincount(y_train)[i]) for i in np.unique(y_train)]))
class_weights

# Define the Random Forest classifier
classifier = RandomForestClassifier(class_weight=class_weights)

# Create the pipeline
pipe_target_enc = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', classifier)
])


# Fit the pipeline to the training data
pipe_target_enc.fit(X_train, y_train)

# Get feature importances
feature_importances = pipe_target_enc.named_steps['classifier'].feature_importances_

# Create a DataFrame to store feature importances
importance_df = pd.DataFrame({'Feature': pipe_target_enc.named_steps['preprocessor'].get_feature_names_out(), 'Importance': feature_importances})

# Sort the DataFrame by importance (descending order)
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Print feature importances
print("Feature Importances:")
for feature, importance in zip(importance_df['Feature'], importance_df['Importance']):
    print(feature, importance)

# Make predictions on the test set
y_pred = pipe_target_enc.predict(X_val)

# Generate the classification report
report = classification_report(y_val, y_pred)
print("Classification Report:")
print(report)

Feature Importances:
target__AgeOrd 0.16715487895636677
target__Tenure 0.1478561817701612
target__NumOfProducts 0.1429802655137449
target__BalanceOrd 0.09961685910765535
target__PointsEarnedQuant 0.09729210965645636
target__SatisfactionScore 0.09558576791667456
target__TaxBracket 0.07210813285378426
target__CountryOrd 0.06217266480062761
target__IsActiveMember 0.04325779157942778
target__HasCrCard 0.032400015971643076
target__GenderBinary 0.029748324247093123
target__SurnameOrd 0.009827007626365078
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.96      0.92       796
           1       0.75      0.51      0.61       204

    accuracy                           0.87      1000
   macro avg       0.82      0.74      0.76      1000
weighted avg       0.86      0.87      0.86      1000



In [54]:

# Define the parameter grid to search over
param_grid = {
    'preprocessor__target__smoothing': [1.0,2, 5, 7],
    'classifier__n_estimators': [250, 300, 350],
    'classifier__max_depth': [6,7,8,9]
}

# pipe_target_enc
# Create the GridSearchCV object
grid_search = GridSearchCV(pipe_onehot_targetEnc, param_grid, cv=5, scoring='f1_macro', verbose=4, n_jobs=-1)

# Fit the GridSearchCV object to the data
# grid_search.fit(X_train, y_train)
grid_search.fit(X_trainDF, y_trainDF)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score (F1 Macro):", grid_search.best_score_)

# Get the best model from the grid search
best_model = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred = best_model.predict(X_testDF)

# Generate the classification report
report = classification_report(y_testDF, y_pred)
print("Classification Report:")
print(report)


Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best Parameters: {'classifier__max_depth': 8, 'classifier__n_estimators': 350, 'preprocessor__target__smoothing': 1.0}
Best Score (F1 Macro): 0.7474072980454554
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.95      0.92      1616
           1       0.71      0.54      0.61       384

    accuracy                           0.87      2000
   macro avg       0.80      0.74      0.77      2000
weighted avg       0.86      0.87      0.86      2000



In [ ]:
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Warning: No categorical columns found. Calling 'transform' will only return input data.
Best Parameters: {'classifier__max_depth': 9, 'classifier__n_estimators': 250, 'preprocessor__target__smoothing': 7}
Best Score (F1 Macro): 0.7433245279184252
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.85      0.88       796
           1       0.55      0.71      0.62       204

    accuracy                           0.82      1000
   macro avg       0.73      0.78      0.75      1000
weighted avg       0.84      0.82      0.83      1000

#! RandomForrest


In [34]:
# Define the categorical features for one-hot encoding
categorical_features = ['Country', 'Gender', 'HasCrCard', 'IsActiveMember','CardType']

# Define the ordinal features for ordinal encoding
ordinal_features = ['Tenure', 'NumOfProducts', 'SatisfactionScore']

# Define the continuous features for scaling
continuous_features = ['CreditScore', 'Age', 'Balance', 'PointsEarned', 'EstimatedSalary']

target_enc_features = ['Tenure', 'NumOfProducts', 'SatisfactionScore', 'Country', 'Gender', 'HasCrCard', 'IsActiveMember','CardType']



# Define the preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), categorical_features),
        ('target', TargetEncoder(), target_enc_features),
        ('scaler', RobustScaler(), continuous_features) # For potential outliers
    ])

# Adjusts weights so that the undersampled class get higher weight
class_weights = dict(zip(np.unique(y_trainDF), [len(y_trainDF) / (len(np.unique(y_trainDF)) * np.bincount(y_trainDF)[i]) for i in np.unique(y_trainDF)]))
class_weights

# Define the Random Forest classifier
classifier = RandomForestClassifier(class_weight=class_weights)

# Create the pipeline
pipe_onehot_targetEnc = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', classifier)
])


# Fit the pipeline to the training data
pipe_onehot_targetEnc.fit(X_trainDF, y_trainDF)

# Get feature importances
feature_importances = pipe_onehot_targetEnc.named_steps['classifier'].feature_importances_

# Create a DataFrame to store feature importances
importance_df = pd.DataFrame({'Feature': pipe_onehot_targetEnc.named_steps['preprocessor'].get_feature_names_out(), 'Importance': feature_importances})

# Sort the DataFrame by importance (descending order)
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Print feature importances
print("Feature Importances:")
for feature, importance in zip(importance_df['Feature'], importance_df['Importance']):
    print(feature, importance)

# Make predictions on the test set
y_pred = pipe_onehot_targetEnc.predict(X_testDF)

# Generate the classification report
report = classification_report(y_testDF, y_pred)
print("Classification Report:")
print(report)


Feature Importances:
scaler__Age 0.2191045913194815
target__NumOfProducts 0.1100858308924759
scaler__Balance 0.10235206891549274
scaler__PointsEarned 0.09242691657510439
scaler__EstimatedSalary 0.09153805606020109
scaler__CreditScore 0.09128327977506823
target__Tenure 0.06039964749107905
target__SatisfactionScore 0.040926670634999426
target__CardType 0.020983991606153003
target__Country 0.01916544736596254
onehot__IsActiveMember_1 0.015067335357439786
onehot__Country_Germany 0.013287458799656073
onehot__IsActiveMember_0 0.013049338055167531
target__IsActiveMember 0.01149530452115938
onehot__CardType_SILVER 0.010055495622611814
onehot__CardType_PLATINUM 0.009360755792184072
onehot__Country_France 0.008683822533543136
onehot__CardType_DIAMOND 0.008604852618898741
target__HasCrCard 0.008329768540221065
onehot__HasCrCard_1 0.008246432219189644
onehot__CardType_GOLD 0.008220944863335305
onehot__HasCrCard_0 0.007864344691387418
target__Gender 0.007723068337277875
onehot__Gender_Female 0.0076

In [ ]:
# Define the LightGBM model

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('onehot', OneHotEncoder(), categorical_features),
#         ('target', TargetEncoder(), target_enc_features),
#         ('scaler', RobustScaler(), continuous_features) # For potential outliers
#     ])


lgm = lgb.LGBMClassifier()
pipe_target_enc.named_steps['classifier'] = lgm
# pipe_onehot_targetEnc = Pipeline([
#     ('preprocessor', preprocessor),
#     ('classifier', model)
# ])


# Define the hyperparameter grid for GridSearchCV
param_grid = {
    # 'num_leaves': [10, 20, 30],
    'max_depth': [3, 5, 7],
}

# Define the scoring metric
scoring = 'accuracy'  # Change to your desired scoring metric

# Perform GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(pipe_target_enc, param_grid, scoring=scoring, cv=5)
grid_search.fit(X_train, y_train)


# Get the best model and its hyperparameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Evaluate the best model on the test set
y_pred = best_model.predict(X_val)
classification_report = classification_report(y_val, y_pred)
print(classification_report)

In [82]:
from sklearn.ensemble import RandomForestClassifier

# Define the parameter values
criteria = ['gini', 'entropy']
max_features = ['sqrt', 'log2']
max_depth = [3, 5, 7]
min_samples_split = [2, 4, 6]
min_samples_leaf = [1, 2, 3]

param = {
    'criterion': criteria,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf
}

# Create an instance of the RandomForestClassifier
rf_cls = RandomForestClassifier()

# Create the GridSearchCV object
grid_search = GridSearchCV(rf_cls, param, cv=5, scoring='f1_macro', verbose=4, n_jobs=-1)

# Fit the GridSearchCV object to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score (F1 Macro):", grid_search.best_score_)

# Get the best model from the grid search
best_model = grid_search.best_estimator_

# Use the best model for prediction
y_pred = best_model.predict(X_val)

# Generate the classification report
report = classification_report(y_val, y_pred)
print('Model:  ' rf_cls.__class__.__name__)
print()
print("Classification Report:")
print(report)



Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'criterion': 'gini', 'max_depth': 7, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 4}
Best Score (F1 Macro): 0.7037421002959607
RandomForestClassifier
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.99      0.93       796
           1       0.89      0.44      0.59       204

    accuracy                           0.87      1000
   macro avg       0.88      0.71      0.76      1000
weighted avg       0.88      0.87      0.86      1000



<font color='Tangerine'> 
Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'criterion': 'gini', 'max_depth': 7, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 4}
Best Score (F1 Macro): 0.7037421002959607
Model name:  RandomForestClassifier
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.99      0.93       796
           1       0.89      0.44      0.59       204

    accuracy                           0.87      1000
   macro avg       0.88      0.71      0.76      1000
weighted avg       0.88      0.87      0.86      1000
</font>

In [74]:


target_enc_features = ['GenderBinary', 'CountryOrd', 'SurnameOrd', 'HasCrCard', 'IsActiveMember', 'Tenure', 'NumOfProducts',
                       'SatisfactionScore', 'PointsEarnedQuant', 'AgeOrd', 'TaxBracket', 'BalanceOrd']


# Increase the weight value in proportion to its frequency to balance a bit for the current undersampling
class_weights = dict(zip(np.unique(y_train), [len(y_train) / (len(np.unique(y_train)) * np.bincount(y_train)[i]) for i in np.unique(y_train)]))


# Create an instance of the LGBMClassifier
lgm = lgb.LGBMClassifier(class_weight=class_weights)

# Define the parameter grid
param_grid = {
    'classifier__num_leaves': [20,25,30],
    'classifier__max_depth': [7,8,9],
    'classifier__learning_rate': [0.05, 0.1,0.2,0.25]
}

preprocessor = ColumnTransformer([
    ('target_encoder', TargetEncoder(), target_enc_features),
])


# Create the pipeline
pipeline2 = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', lgm)
])

# Create the GridSearchCV object
grid_search = GridSearchCV(pipeline2, param_grid, cv=5, scoring='f1_macro', verbose=4, n_jobs=-1)

# Fit the GridSearchCV object to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score (F1 Macro):", grid_search.best_score_)

# Get the best model from the grid search
best_model = grid_search.best_estimator_


y_pred = best_model.predict(X_val)

report = classification_report(y_val, y_pred)
print()
print("Classification Report:")
print(report)


Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best Parameters: {'classifier__learning_rate': 0.1, 'classifier__max_depth': 7, 'classifier__num_leaves': 30}
Best Score (F1 Macro): 0.7286351717896473

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.79      0.86       796
           1       0.48      0.76      0.59       204

    accuracy                           0.79      1000
   macro avg       0.71      0.78      0.72      1000
weighted avg       0.84      0.79      0.80      1000



<font color='Tangerine'> 
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Warning: No categorical columns found. Calling 'transform' will only return input data.
Best Parameters: {'classifier__learning_rate': 0.1, 'classifier__max_depth': 7, 'classifier__num_leaves': 20}
Best Score (F1 Macro): 0.7398664837604242

Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.96      0.92       796
           1       0.78      0.54      0.64       204

    accuracy                           0.88      1000
   macro avg       0.84      0.75      0.78      1000
weighted avg       0.87      0.88      0.87      1000
</font>

In [14]:
target_enc_features = ['GenderBinary', 'CountryOrd', 'SurnameOrd', 'HasCrCard', 'IsActiveMember', 'Tenure', 'NumOfProducts',
                       'SatisfactionScore', 'PointsEarnedQuant', 'AgeOrd', 'TaxBracket', 'BalanceOrd']


# Increase the weight value in proportion to its frequency to balance a bit for the current undersampling
class_weights = dict(zip(np.unique(y_train), [len(y_train) / (len(np.unique(y_train)) * np.bincount(y_train)[i]) for i in np.unique(y_train)]))


# Create an instance of the LGBMClassifier
logreg = LogisticRegression(class_weight=class_weights) # class_weight=class_weights

# Define the parameter grid
param_grid = {
    'classifier__C': [0.1, 1.0, 10.0],
    'classifier__penalty': ['l2']
}

preprocessor = ColumnTransformer([
    ('target_encoder', TargetEncoder(), target_enc_features),
])


# Create the pipeline
pipe_logReg = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', logreg)
])

# Create the GridSearchCV object
grid_search = GridSearchCV(pipe_logReg, param_grid, cv=5, scoring='f1_macro', verbose=4, n_jobs=-1)

# Fit the GridSearchCV object to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score (F1 Macro):", grid_search.best_score_)

# Get the best model from the grid search
best_model = grid_search.best_estimator_


y_pred = best_model.predict(X_val)

report = classification_report(y_val, y_pred)
print()
print("Classification Report:")
print(report)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'classifier__C': 1.0, 'classifier__penalty': 'l2'}
Best Score (F1 Macro): 0.6389185818926236

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.69      0.78       796
           1       0.36      0.70      0.48       204

    accuracy                           0.69      1000
   macro avg       0.63      0.69      0.63      1000
weighted avg       0.79      0.69      0.72      1000



<font color='Tangerine'> 
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Warning: No categorical columns found. Calling 'transform' will only return input data.
Best Parameters: {'classifier__C': 1.0, 'classifier__penalty': 'l2'}
Best Score (F1 Macro): 0.6389185818926236

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.71      0.80       796
           1       0.40      0.75      0.52       204

    accuracy                           0.72      1000
   macro avg       0.66      0.73      0.66      1000
weighted avg       0.81      0.72      0.74      1000

</font>

In [15]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

target_enc_features = ['GenderBinary', 'CountryOrd', 'SurnameOrd', 'HasCrCard', 'IsActiveMember', 'Tenure', 'NumOfProducts',
                       'SatisfactionScore', 'PointsEarnedQuant', 'AgeOrd', 'TaxBracket', 'BalanceOrd']

# Increase the weight value in proportion to its frequency to balance a bit for the current undersampling
class_weights = dict(zip(np.unique(y_train), [len(y_train) / (len(np.unique(y_train)) * np.bincount(y_train)[i]) for i in np.unique(y_train)]))

# Create an instance of the XGBClassifier
xgb_cls = XGBClassifier()

# Define the parameter grid
param_grid = {
    'classifier__learning_rate': [0.05, 0.1, 0.15],
    'classifier__max_depth': [3, 5, 7,9],
    'classifier__n_estimators': [150, 200, 300]
}

preprocessor = ColumnTransformer([
    ('target_encoder', TargetEncoder(), target_enc_features),
])

# Create the pipeline
pipe_xgb_cls = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', xgb_cls)
])

# Create the GridSearchCV object
grid_search = GridSearchCV(pipe_xgb_cls, param_grid, cv=7, scoring='f1_macro', verbose=4, n_jobs=-1)

# Fit the GridSearchCV object to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score (F1 Macro):", grid_search.best_score_)

# Get the best model from the grid search
best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_val)

report = classification_report(y_val, y_pred)
print()
print("Classification Report:")
print(report)



Fitting 7 folds for each of 36 candidates, totalling 252 fits
Best Parameters: {'classifier__learning_rate': 0.05, 'classifier__max_depth': 7, 'classifier__n_estimators': 200}
Best Score (F1 Macro): 0.7387595084567579

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.95      0.91       796
           1       0.71      0.45      0.55       204

    accuracy                           0.85      1000
   macro avg       0.79      0.70      0.73      1000
weighted avg       0.84      0.85      0.84      1000



<font color='Tangerine'> 
Fitting 7 folds for each of 36 candidates, totalling 252 fits
Warning: No categorical columns found. Calling 'transform' will only return input data.
Best Parameters: {'classifier__learning_rate': 0.05, 'classifier__max_depth': 7, 'classifier__n_estimators': 200}
Best Score (F1 Macro): 0.7387595084567579

Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.96      0.92       796
           1       0.76      0.53      0.63       204

    accuracy                           0.87      1000
   macro avg       0.83      0.75      0.78      1000
weighted avg       0.86      0.87      0.86      1000
</font>

In [16]:
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report

target_enc_features = ['GenderBinary', 'CountryOrd', 'SurnameOrd', 'HasCrCard', 'IsActiveMember', 'Tenure', 'NumOfProducts',
                       'SatisfactionScore', 'PointsEarnedQuant', 'AgeOrd', 'TaxBracket', 'BalanceOrd']

# Increase the weight value in proportion to its frequency to balance a bit for the current undersampling
class_weights = dict(zip(np.unique(y_train), [len(y_train) / (len(np.unique(y_train)) * np.bincount(y_train)[i]) for i in np.unique(y_train)]))

# Create an instance of the CatBoostClassifier
catboost_cls = CatBoostClassifier()

# Define the parameter grid
param_grid = {
    'classifier__learning_rate': [0.05, 0.1, 0.15],
    'classifier__depth': [3, 5, 7, 9],
    'classifier__n_estimators': [150, 200, 300]
}

preprocessor = ColumnTransformer([
    ('target_encoder', TargetEncoder(), target_enc_features),
])

# Create the pipeline
pipe_catboost_cls = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', catboost_cls)
])

# Create the GridSearchCV object
grid_search = GridSearchCV(pipe_catboost_cls, param_grid, cv=7, scoring='f1_macro', verbose=4, n_jobs=-1)

# Fit the GridSearchCV object to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score (F1 Macro):", grid_search.best_score_)

# Get the best model from the grid search
best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_val)

report = classification_report(y_val, y_pred)
print(catboost_cls.__class__.__name__)
print()
print("Classification Report:")
print(report)


Fitting 7 folds for each of 36 candidates, totalling 252 fits
0:	learn: 0.6216110	total: 56.2ms	remaining: 8.37s
1:	learn: 0.5657334	total: 58.3ms	remaining: 4.32s
2:	learn: 0.5240311	total: 60.5ms	remaining: 2.97s
3:	learn: 0.4927936	total: 62.6ms	remaining: 2.28s
4:	learn: 0.4672428	total: 64.8ms	remaining: 1.88s
5:	learn: 0.4463725	total: 68.4ms	remaining: 1.64s
6:	learn: 0.4290862	total: 70.7ms	remaining: 1.45s
7:	learn: 0.4162892	total: 73ms	remaining: 1.29s
8:	learn: 0.4040249	total: 75.1ms	remaining: 1.18s
9:	learn: 0.3944090	total: 77.6ms	remaining: 1.09s
10:	learn: 0.3848699	total: 79.8ms	remaining: 1.01s
11:	learn: 0.3793010	total: 81.9ms	remaining: 942ms
12:	learn: 0.3737354	total: 84.5ms	remaining: 891ms
13:	learn: 0.3693152	total: 87.6ms	remaining: 851ms
14:	learn: 0.3648079	total: 89.8ms	remaining: 808ms
15:	learn: 0.3607570	total: 92.3ms	remaining: 773ms
16:	learn: 0.3591487	total: 93.8ms	remaining: 734ms
17:	learn: 0.3565105	total: 96.2ms	remaining: 705ms
18:	learn: 0.3

<font color='Tangerine'> 
Best Parameters: {'classifier__depth': 7, 'classifier__learning_rate': 0.15, 'classifier__n_estimators': 150}
Best Score (F1 Macro): 0.7417607832161796

Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.96      0.92       796
           1       0.77      0.54      0.64       204

    accuracy                           0.87      1000
   macro avg       0.83      0.75      0.78      1000
weighted avg       0.87      0.87      0.87      1000

</font>

In [30]:
X.categorical_features.value_counts()

AttributeError: 'DataFrame' object has no attribute 'categorical_features'

In [44]:
print(X.shape)
print(X_val_cluster.shape)
print(X_test_cluster.shape)

(8000, 21)
(1000, 21)
(1000, 21)


In [37]:
X.head()

,Tenure,NumOfProducts,HasCrCard,IsActiveMember,SatisfactionScore,GenderBinary,SurnameOrd,CardTypeOrd,CountryOrd,CreditScore,...,Balance,EstimatedSalary,PointsEarned,TenurePoints,CardProducts,SatisfactionProducts,AgeProducts,BalanceCredit,ClusterKMeans,ClusterBGM
0,2,2,1,0,2,0,0.0,1.0,1,709,...,0.00,104982.39,422,0.004739,0.5,1.0,17.5,0.000000,1,3
1,1,1,1,1,1,0,0.0,2.0,2,744,...,43504.42,119327.75,607,0.001647,2.0,1.0,29.0,58.473683,0,2
2,2,1,0,1,1,0,0.0,0.0,2,773,...,145578.28,186172.85,630,0.003175,0.0,1.0,64.0,188.328952,0,2
3,4,1,1,0,1,1,2.0,2.0,3,646,...,105957.44,15470.91,345,0.011594,2.0,1.0,29.0,164.020805,0,2
4,8,2,0,1,3,1,0.0,0.0,2,675,...,0.00,95463.29,632,0.012658,0.0,1.5,28.5,0.000000,1,3


In [38]:
X.columns

Index(['Tenure', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'SatisfactionScore', 'GenderBinary', 'SurnameOrd', 'CardTypeOrd',
       'CountryOrd', 'CreditScore', 'Age', 'Balance', 'EstimatedSalary',
       'PointsEarned', 'TenurePoints', 'CardProducts', 'SatisfactionProducts',
       'AgeProducts', 'BalanceCredit', 'ClusterKMeans', 'ClusterBGM'],
      dtype='object')

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), categorical_features),
        ('target', TargetEncoder(), target_enc_features),
        ('scaler', RobustScaler(), continuous_features) # For potential outliers
    ])

In [46]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report



continuous_features = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary', 'PointsEarned']

categorical_features = ['HasCrCard', 'IsActiveMember', 'GenderBinary']

cluster_features = ['ClusterKMeans', 'ClusterBGM']

ordinal_features = ['Tenure', 'CardTypeOrd', 'NumOfProducts', 'NumOfProducts']

target_enc_features = ['GenderBinary', 'CountryOrd', 'SurnameOrd', 'HasCrCard', 'IsActiveMember',  'Tenure', 'NumOfProducts','SatisfactionScore']

# Increase the weight value in proportion to its frequency to balance a bit for the current undersampling
class_weights = dict(zip(np.unique(y_train), [len(y_train) / (len(np.unique(y_train)) * np.bincount(y_train)[i]) for i in np.unique(y_train)]))

# Create an instance of the XGBClassifier
xgb_cls = XGBClassifier()

# Define the parameter grid
param_grid = {
    'classifier__learning_rate': [0.05, 0.1, 0.15],
    'classifier__max_depth': [3, 5, 7,9],
    'classifier__n_estimators': [150, 200, 300]
}

prep = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), categorical_features),
        ('target', TargetEncoder(), target_enc_features),
        ('scaler', StandardScaler(), continuous_features),
        ('ordinal', 'passthrough', ordinal_features),
        ('cluster', 'passthrough', cluster_features),
    ])



# Create the pipeline
pipe_xgb_cls = Pipeline([
    ('preprocessor', prep),
    ('classifier', xgb_cls)
])

# Create the GridSearchCV object
grid_search = GridSearchCV(pipe_xgb_cls, param_grid, cv=7, scoring='f1_macro', verbose=4, n_jobs=-1)

# Fit the GridSearchCV object to the data
grid_search.fit(X, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score (F1 Macro):", grid_search.best_score_)

# Get the best model from the grid search
best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_val_cluster)

report = classification_report(y_val, y_pred)
print()
print("Classification Report:")
print(report)

Fitting 7 folds for each of 36 candidates, totalling 252 fits
Best Parameters: {'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__n_estimators': 300}
Best Score (F1 Macro): 0.750233730665283

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.95      0.91       796
           1       0.70      0.48      0.57       204

    accuracy                           0.85      1000
   macro avg       0.79      0.71      0.74      1000
weighted avg       0.84      0.85      0.84      1000



<font color='Tangerine'> 
Fitting 7 folds for each of 36 candidates, totalling 252 fits
Warning: No categorical columns found. Calling 'transform' will only return input data.
Best Parameters: {'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__n_estimators': 300}
Best Score (F1 Macro): 0.750233730665283

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.95      0.91       796
           1       0.70      0.48      0.57       204

    accuracy                           0.85      1000
   macro avg       0.79      0.71      0.74      1000
weighted avg       0.84      0.85      0.84      1000
</font>

In [53]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from category_encoders import TargetEncoder

# Define the preprocessing steps for different types of columns
preprocessor = ColumnTransformer(transformers=[
    ('onehot', OneHotEncoder(), ['HasCrCard', 'IsActiveMember', 'GenderBinary']),
    ('target', TargetEncoder(), ['GenderBinary', 'CountryOrd', 'SurnameOrd', 'HasCrCard', 'IsActiveMember', 'Tenure', 'NumOfProducts', 'SatisfactionScore']),
    ('scaler', StandardScaler(), ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary', 'PointsEarned']),
    ('ordinal', 'passthrough', ['Tenure', 'CardTypeOrd', 'NumOfProducts', 'NumOfProducts']),
    ('cluster', 'passthrough', ['ClusterKMeans', 'ClusterBGM'])
])

# Define the classifier
classifier = RandomForestClassifier()

# Create the pipeline
pipe_rf_cls = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', classifier)
])

# Define the parameter grid
param = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [None, 5, 10],
    'classifier__min_samples_split': [2, 5, 10],
}

# Perform grid search
grid_search = GridSearchCV(pipe_rf_cls, param, cv=5, scoring='f1_macro', verbose=4, n_jobs=-1)

# Fit the GridSearchCV object to the data
grid_search.fit(X, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)

# Get the best model from the grid search
best_model = grid_search.best_estimator_

# Use the best model for prediction
y_pred = best_model.predict(X_val_cluster)

# Generate the classification report
report = classification_report(y_val, y_pred)
print()
print("Classification Report:")
print(report)


Fitting 5 folds for each of 27 candidates, totalling 135 fits
Best Parameters: {'classifier__max_depth': None, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 200}

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.96      0.91       796
           1       0.75      0.46      0.57       204

    accuracy                           0.86      1000
   macro avg       0.81      0.71      0.74      1000
weighted avg       0.85      0.86      0.84      1000



<font color='Tangerine'> 
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Warning: No categorical columns found. Calling 'transform' will only return input data.
Best Parameters: {'classifier__max_depth': None, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 200}

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.96      0.91       796
           1       0.75      0.46      0.57       204

    accuracy                           0.86      1000
   macro avg       0.81      0.71      0.74      1000
weighted avg       0.85      0.86      0.84      1000
</font>

In [54]:
X.columns

Index(['Tenure', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'SatisfactionScore', 'GenderBinary', 'SurnameOrd', 'CardTypeOrd',
       'CountryOrd', 'CreditScore', 'Age', 'Balance', 'EstimatedSalary',
       'PointsEarned', 'TenurePoints', 'CardProducts', 'SatisfactionProducts',
       'AgeProducts', 'BalanceCredit', 'ClusterKMeans', 'ClusterBGM'],
      dtype='object')

In [62]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

#* With semi duplicate features i.e. Balance AND BalanceCredit

continuous_features = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary', 'PointsEarned', 'TenurePoints', 'CardProducts', 'SatisfactionProducts', 'AgeProducts', 'BalanceCredit']

categorical_features = ['HasCrCard', 'IsActiveMember', 'GenderBinary']

cluster_features = ['ClusterKMeans', 'ClusterBGM']

ordinal_features = ['Tenure', 'CardTypeOrd', 'NumOfProducts', 'NumOfProducts']

target_enc_features = ['GenderBinary', 'CountryOrd', 'SurnameOrd', 'HasCrCard', 'IsActiveMember',  'Tenure', 'NumOfProducts','SatisfactionScore', 'TenurePoints', 'CardProducts', 'SatisfactionProducts', 'AgeProducts', 'BalanceCredit']

# Increase the weight value in proportion to its frequency to balance a bit for the current undersampling
class_weights = dict(zip(np.unique(y_train), [len(y_train) / (len(np.unique(y_train)) * np.bincount(y_train)[i]) for i in np.unique(y_train)]))

# Create an instance of the XGBClassifier
xgb_cls = XGBClassifier()

# Define the parameter grid
param_grid = {
    'classifier__learning_rate': [0.05, 0.1, 0.15],
    'classifier__max_depth': [3, 5, 7,9],
    'classifier__n_estimators': [150, 200, 300]
}

prep = ColumnTransformer(
    transformers=[
        ('scaler', StandardScaler(), continuous_features),
        ('ordinal', 'passthrough', ordinal_features),
        ('cluster', 'passthrough', cluster_features),
        ('categorical', 'passthrough', categorical_features)      
    ])



# Create the pipeline
pipe_xgb_cls = Pipeline([
    ('preprocessor', prep),
    ('classifier', xgb_cls)
])

# Create the GridSearchCV object
grid_search_xg = GridSearchCV(pipe_xgb_cls, param_grid, cv=3, scoring='f1_macro', verbose=4, n_jobs=-1)

# Fit the GridSearchCV object to the data
grid_search_xg.fit(X, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search_xg.best_params_)
print("Best Score (F1 Macro):", grid_search_xg.best_score_)

# Get the best model from the grid search
best_model = grid_search_xg.best_estimator_

y_pred = best_model.predict(X_val_cluster)

report = classification_report(y_val, y_pred)
print()
print("Classification Report:")
print(report)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best Parameters: {'classifier__learning_rate': 0.05, 'classifier__max_depth': 5, 'classifier__n_estimators': 150}
Best Score (F1 Macro): 0.7315665204230263

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.96      0.91       796
           1       0.75      0.45      0.56       204

    accuracy                           0.86      1000
   macro avg       0.81      0.70      0.74      1000
weighted avg       0.85      0.86      0.84      1000



In [59]:
#* Bummer. The feature engineering did not improve the resutls

continuous_features = ['EstimatedSalary', 'TenurePoints', 'CardProducts', 'SatisfactionProducts', 'AgeProducts', 'BalanceCredit']

categorical_features = ['HasCrCard', 'IsActiveMember', 'GenderBinary']

cluster_features = ['ClusterKMeans', 'ClusterBGM']

ordinal_features = []

target_enc_features = ['CountryOrd', 'TenurePoints', 'SatisfactionProducts', 'AgeProducts', 'BalanceCredit']

# Increase the weight value in proportion to its frequency to balance a bit for the current undersampling
class_weights = dict(zip(np.unique(y_train), [len(y_train) / (len(np.unique(y_train)) * np.bincount(y_train)[i]) for i in np.unique(y_train)]))

# Create an instance of the XGBClassifier
xgb_cls = XGBClassifier()

# Define the parameter grid
param_grid = {
    'classifier__learning_rate': [0.05, 0.1, 0.15],
    'classifier__max_depth': [3, 5, 7,9],
    'classifier__n_estimators': [150, 200, 300]
}

prep = ColumnTransformer(
    transformers=[
        # ('onehot', OneHotEncoder(), categorical_features),
        ('target', TargetEncoder(), target_enc_features),
        ('scaler', StandardScaler(), continuous_features),
        ('ordinal', 'passthrough', ordinal_features),
        ('cluster', 'passthrough', cluster_features),
        ('categorical', 'passthrough', categorical_features)
        
    ])



# Create the pipeline
pipe_xgb_cls = Pipeline([
    ('preprocessor', prep),
    ('classifier', xgb_cls)
])

# Create the GridSearchCV object
grid_search_xg222 = GridSearchCV(pipe_xgb_cls, param_grid, cv=3, scoring='f1_macro', verbose=4, n_jobs=-1)

# Fit the GridSearchCV object to the data
grid_search_xg222.fit(X, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search_xg222.best_params_)
print("Best Score (F1 Macro):", grid_search_xg222.best_score_)

# Get the best model from the grid search
best_model = grid_search_xg222.best_estimator_

y_pred = best_model.predict(X_val_cluster)

report = classification_report(y_val, y_pred)
print()
print("Classification Report:")
print(report)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best Parameters: {'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__n_estimators': 300}
Best Score (F1 Macro): 0.7385244777083596

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.96      0.91       796
           1       0.73      0.47      0.57       204

    accuracy                           0.86      1000
   macro avg       0.80      0.71      0.74      1000
weighted avg       0.85      0.86      0.84      1000



<font color='Tangerine'> 
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Warning: No categorical columns found. Calling 'transform' will only return input data.
Best Parameters: {'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__n_estimators': 300}
Best Score (F1 Macro): 0.7385244777083596

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.96      0.91       796
           1       0.73      0.47      0.57       204

    accuracy                           0.86      1000
   macro avg       0.80      0.71      0.74      1000
weighted avg       0.85      0.86      0.84      1000
</font>

In [57]:
X.columns

Index(['Tenure', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'SatisfactionScore', 'GenderBinary', 'SurnameOrd', 'CardTypeOrd',
       'CountryOrd', 'CreditScore', 'Age', 'Balance', 'EstimatedSalary',
       'PointsEarned', 'TenurePoints', 'CardProducts', 'SatisfactionProducts',
       'AgeProducts', 'BalanceCredit', 'ClusterKMeans', 'ClusterBGM'],
      dtype='object')

In [ ]:
max_thresold = df['salary_in_usd'].quantile(0.98)
max_thresold
print(max_thresold) 


# df[df['salary_in_usd']>max_thresold]

min_thresold = df['salary_in_usd'].quantile(0.03)
min_thresold

df[df['salary_in_usd']<min_thresold]

print(len(df))

df = df[(df['salary_in_usd']<max_thresold) & (df['salary_in_usd']>min_thresold)]

print(len(df))